In [12]:
from numba import njit, jit
import matplotlib.pyplot as plt
import numpy as np
import time
import scipy
import typing
import math
from scipy.stats import binom, multinomial

# シード値を設定
seed = 0
np.random.seed(seed)

# 初期パラメータを設定
num_constraints = 3  # 制約の数
num_qubits = 4     # キュービットの数
p = 1                # QAOAの層の数

# まずは、新しいproxyのデータを生成する。

# (start_time, start_value) と (end_time, end_value) の間の直線上で、current_time における y 値を計算
@njit
def line_between(current_time: float, start_time: float, start_value: float, end_time: float, end_value: float) -> float:
    # current_time が start_time から end_time までの間のどこにあるかを計算
    relative_time = (current_time - start_time) / (end_time - start_time)

    # relative_time が 0 から 1 に変化するにつれて、start_value から end_value まで変化。
    return (1 - relative_time) * start_value + relative_time * end_value

# 三角形の曲線上で、与えられた x の位置に対応する y 値を返す
@njit
def triangle_value(x: int, left: int, right: int, height: float) -> float:
    return max(0, min(x - left, right - x) * 2 * height / (right - left))

# コスト c に対する確率 P(c') を計算
@njit
def prob_cost(cost: int, num_constraints: int) -> float:
    return 4 / ((num_constraints + 1) ** 2) * min(cost + 1, num_constraints + 1 - cost)

# コスト c に対する数 N(c') を計算
@njit
def number_with_cost_proxy(cost: int, num_constraints: int, num_qubits: int) -> float:
    scale = 1 << num_qubits
    return prob_cost(cost, num_constraints) * scale

# 距離 d におけるコスト c1 から c2 への遷移の数 N(c'; d, c) を近似的に計算
# TODO: これは prob_edge = 0.5 の場合にのみ機能
@njit
def number_of_costs_at_distance_proxy(cost_1: int, cost_2: int, distance: int, num_constraints: int, num_qubits: int) -> float:
    # 距離が 0 から num_qubits//2 の間にあることを確認
    reflected_distance = distance
    if distance > num_qubits // 2:
        reflected_distance = num_qubits - distance

    # 多項分布のピーク値を近似
    h_peak = 1 << (num_qubits - 4)
    # (0 または num_qubits, 1) と (num_qubits/2, h_peak) の直線上で→
    # reflected_distance におけるピークの高さを計算
    h_at_cost_2 = line_between(reflected_distance, 0, 1, num_qubits / 2, h_peak)
    # cost_1 と num_constraints/2 の直線上で→
    # cost_2 におけるピークの高さを計算
    center = line_between(reflected_distance, 0, cost_1, num_qubits / 2, num_constraints / 2)
    left = center - reflected_distance - 1
    right = center + reflected_distance + 1

    return triangle_value(cost_2, left, right, h_at_cost_2)

# アルゴリズム1のforループ内の和を計算
@njit
def compute_amplitude_sum(prev_amplitudes: np.ndarray, gamma: float, beta: float, cost_1: int, num_constraints: int, num_qubits: int) -> complex:
    sum = 0
    for cost_2 in range(num_constraints + 1):
        for distance in range(num_qubits + 1):
            beta_factor = (np.cos(beta) ** (num_qubits - distance)) * ((-1j * np.sin(beta)) ** distance)
            gamma_factor = np.exp(-1j * gamma * cost_2)
            num_costs_at_distance = number_of_costs_at_distance_proxy(cost_1, cost_2, distance, num_constraints, num_qubits)
            sum += beta_factor * gamma_factor * prev_amplitudes[cost_2] * num_costs_at_distance
    return sum

# 代理の期待値を最適化する代わりに、高コスト振幅を最適化する (例: 指数重み付けを使用して)
# アルゴリズム1を代理の近似を使用して実装
# num_constraints = エッジの数、num_qubits = 頂点の数
@njit
def QAOA_proxy(p: int, gamma: np.ndarray, beta: np.ndarray, num_constraints: int, num_qubits: int):
    num_costs = num_constraints + 1
    amplitude_proxies = np.zeros((p + 1, num_costs), dtype=np.complex128)
    init_amplitude = np.sqrt(1 / (1 << num_qubits))
    for i in range(num_costs):
        amplitude_proxies[0][i] = init_amplitude

    for current_depth in range(1, p + 1):
        for cost_1 in range(num_costs):
            amplitude_proxies[current_depth][cost_1] = compute_amplitude_sum(
                amplitude_proxies[current_depth - 1], gamma[current_depth - 1], beta[current_depth - 1], cost_1, num_constraints, num_qubits
            )

    expected_proxy = 0
    for cost in range(num_costs):
        expected_proxy += number_with_cost_proxy(cost, num_constraints, num_qubits) * (abs(amplitude_proxies[p][cost]) ** 2) * cost

    return amplitude_proxies, expected_proxy

# 逆代理目的関数を作成
def inverse_proxy_objective_function(num_constraints: int, num_qubits: int, p: int, expectations: list[np.ndarray] | None) -> typing.Callable:
    def inverse_objective(*args) -> float:
        gamma, beta = args[0][:p], args[0][p:]
        _, expectation = QAOA_proxy(p, gamma, beta, num_constraints, num_qubits)
        current_time = time.time()

        if expectations is not None:
            expectations.append((current_time, expectation))

        return -expectation

    return inverse_objective

# 現在は prob_edge = 0.5 の場合にのみ実装
def QAOA_proxy_run(
    num_constraints: int,
    num_qubits: int,
    p: int,
    init_gamma: np.ndarray,
    init_beta: np.ndarray,
    optimizer_method: str = "COBYLA",
    optimizer_options: dict | None = None,
    expectations: list[np.ndarray] | None = None,
) -> dict:
    init_freq = np.hstack([init_gamma, init_beta])

    start_time = time.time()
    result = scipy.optimize.minimize(
        inverse_proxy_objective_function(num_constraints, num_qubits, p, expectations), init_freq, args=(), method=optimizer_method, options=optimizer_options
    )
    # 上記は scipy の最適化結果オブジェクトを返す。このオブジェクトには複数の属性がある
    # result.x ：最適解
    # result.success ：アルゴリズムが成功したかどうかを示すブール値
    # result.message ：アルゴリズムが終了した理由のメッセージ
    # result.nfev ：使用された反復回数（ここではQAOA呼び出し回数）
    end_time = time.time()

    def make_time_relative(input: tuple[float, float]) -> tuple[float, float]:
        time, x = input
        return (time - start_time, x)

    if expectations is not None:
        expectations = list(map(make_time_relative, expectations))

    gamma, beta = result.x[:p], result.x[p:]
    _, expectation = QAOA_proxy(p, gamma, beta, num_constraints, num_qubits)

    return {
        "gamma": gamma,
        "beta": beta,
        "expectation": expectation, #最終的な期待値
        "runtime": end_time - start_time,  # 秒単位での実行時間
        "num_QAOA_calls": result.nfev,  # 反復回数
        "expectations": expectations,  # すべてのQAOA呼び出しに対する期待値
        "scipy_opt_message": result.message,
    }

# 例として、example_expectationsを初期化
example_expectations = []

# QAOA_proxy_runを実行し、結果を変数に代入
result = QAOA_proxy_run(num_constraints, num_qubits, p, init_gamma, init_beta, expectations=example_expectations)

# リストとして保存
new_proxy_gamma = list(result["gamma"])
new_proxy_beta = list(result["beta"])
new_proxy_expectation = result["expectation"]
new_proxy_runtime = result["runtime"]
new_proxy_num_QAOA_calls = result["num_QAOA_calls"]
new_proxy_expectations = result["expectations"]


#つづいて論文に書かれているproxyに基づいて計算する
# 論文におけるP(c')の計算
def prob_cost_paper(cost: int, num_constraints: int, prob_edge: float = 0.5) -> float:
    return binom.pmf(cost, num_constraints, prob_edge)


# 論文におけるN(c')の計算
def number_with_cost_paper_proxy(cost: int, num_constraints: int, num_qubits: int, prob_edge: float = 0.5) -> float:
    scale = 1 << num_qubits  # 2のnum_qubits乗
    return prob_cost_paper(cost, num_constraints, prob_edge) * scale


# 論文におけるP(b, c'-b, c-b | d)の計算
def prob_common_at_distance_paper(num_constraints: int, common_constraints: int, cost_1: int, cost_2: int, distance: int) -> float:
    # 同じ制約がある確率
    prob_same = (math.comb(num_constraints - distance, 2) + math.comb(distance, 2)) / math.comb(num_constraints, 2)
    prob_neither = prob_same / 2
    prob_both = prob_neither
    prob_one = (1 - prob_neither - prob_both) / 2
    return multinomial.pmf(
        [common_constraints, cost_1 - common_constraints, cost_2 - common_constraints, num_constraints + common_constraints - (cost_1 + cost_2)],
        num_constraints,
        [prob_both, prob_one, prob_one, prob_neither],
    )


# 論文におけるN(c'; d, c)の計算
def number_of_costs_at_distance_paper_proxy(cost_1: int, cost_2: int, distance: int, num_constraints: int, num_qubits: int, prob_edge: float = 0.5) -> float:
    sum = 0
    # 共通の制約数の範囲を設定し、その範囲内での確率を計算
    for common_constraints in range(max(0, cost_1 + cost_2 - num_constraints), min(cost_1, cost_2) + 1):
        sum += prob_common_at_distance_paper(num_constraints, common_constraints, cost_1, cost_2, distance)

    p_cost = prob_cost_paper(cost_1, num_constraints, prob_edge)
    return (math.comb(num_qubits, distance) / p_cost) * sum


# 論文のアルゴリズム1のループ内での合計計算
def compute_amplitude_sum_paper(prev_amplitudes: np.ndarray, gamma: float, beta: float, cost_1: int, num_constraints: int, num_qubits: int) -> complex:
    sum = 0
    # cost_2とdistanceの範囲でのループ
    for cost_2 in range(num_constraints + 1):
        for distance in range(num_qubits + 1):
            # βとγのファクターを計算
            beta_factor = (np.cos(beta) ** (num_qubits - distance)) * ((-1j * np.sin(beta)) ** distance)
            gamma_factor = np.exp(-1j * gamma * cost_2)
            num_costs_at_distance = number_of_costs_at_distance_paper_proxy(cost_1, cost_2, distance, num_constraints, num_qubits)
            sum += beta_factor * gamma_factor * prev_amplitudes[cost_2] * num_costs_at_distance
    return sum


# TODO: 期待値プロキシの最適化ではなく、高コスト振幅の最適化を行う場合の検討
# 論文のアルゴリズム1
# num_constraintsはエッジの数、num_qubitsは頂点の数
def QAOA_paper_proxy(p: int, gamma: np.ndarray, beta: np.ndarray, num_constraints: int, num_qubits: int):
    num_costs = num_constraints + 1
    amplitude_proxies = np.zeros([p + 1, num_costs], dtype=complex)
    init_amplitude = np.sqrt(1 / (1 << num_qubits))  # 初期振幅の計算
    for i in range(num_costs):
        amplitude_proxies[0][i] = init_amplitude

    # 各深さでの振幅を計算
    for current_depth in range(1, p + 1):
        for cost_1 in range(num_costs):
            amplitude_proxies[current_depth][cost_1] = compute_amplitude_sum_paper(
                amplitude_proxies[current_depth - 1], gamma[current_depth - 1], beta[current_depth - 1], cost_1, num_constraints, num_qubits
            )

    expected_proxy = 0
    # 最終深さでの期待値を計算
    for cost in range(num_costs):
        expected_proxy += number_with_cost_paper_proxy(cost, num_constraints, num_qubits) * (abs(amplitude_proxies[p][cost]) ** 2) * cost

    return amplitude_proxies, expected_proxy


# 逆問題の目的関数を生成
def inverse_paper_proxy_objective_function(num_constraints: int, num_qubits: int, p: int, expectations: list[np.ndarray] | None) -> typing.Callable:
    def inverse_objective(*args) -> float:
        gamma, beta = args[0][:p], args[0][p:]
        _, expectation = QAOA_paper_proxy(p, gamma, beta, num_constraints, num_qubits)
        current_time = time.time()

        if expectations is not None:
            expectations.append((current_time, expectation))

        return -expectation

    return inverse_objective


# QAOAの実行と最適化
def QAOA_paper_proxy_run(
    num_constraints: int,
    num_qubits: int,
    p: int,
    init_gamma: np.ndarray,
    init_beta: np.ndarray,
    optimizer_method: str = "COBYLA",
    optimizer_options: dict | None = None,
    expectations: list[np.ndarray] | None = None,
) -> dict:
    init_freq = np.hstack([init_gamma, init_beta])  # 初期のγとβを結合

    start_time = time.time()
    result = scipy.optimize.minimize(
        inverse_paper_proxy_objective_function(num_constraints, num_qubits, p, expectations),
        init_freq,
        args=(),
        method=optimizer_method,
        options=optimizer_options,
    )
    end_time = time.time()

    # 経過時間を相対的に計算する関数
    def make_time_relative(input: tuple[float, float]) -> tuple[float, float]:
        time, x = input
        return (time - start_time, x)

    if expectations is not None:
        expectations = list(map(make_time_relative, expectations))

    gamma, beta = result.x[:p], result.x[p:]
    _, expectation = QAOA_paper_proxy(p, gamma, beta, num_constraints, num_qubits)

    return {
        "gamma": gamma,
        "beta": beta,
        "expectation": expectation,
        "runtime": end_time - start_time,  # 経過時間（秒単位）
        "num_QAOA_calls": result.nfev,  # QAOAの呼び出し回数
        "classical_opt_success": result.success,
        "scipy_opt_message": result.message,
    }

# 例として、example_expectationsを初期化
example_expectations = []

# QAOA_proxy_runを実行し、結果を変数に代入
result = QAOA_proxy_run(num_constraints, num_qubits, p, init_gamma, init_beta, expectations=example_expectations)

# リストとして保存
paper_proxy_gamma = list(result["gamma"])
paper_proxy_beta = list(result["beta"])
paper_proxy_expectation = result["expectation"]
paper_proxy_runtime = result["runtime"]
paper_proxy_num_QAOA_calls = result["num_QAOA_calls"]
paper_proxy_expectations = result["expectations"]


##############

# 初期値の範囲とステップサイズを設定
gamma_beta_range = np.arange(0, 5)  # [0, 1, ..., 5]
step_size = 1  # ステップサイズ（範囲内の全組み合わせを探索するため、1に設定）

# 結果を保存するためのリストを初期化
all_results = []

# gammaとbetaの初期値を全ての組み合わせで生成
for gamma_values in np.nditer(np.meshgrid(gamma_beta_range, gamma_beta_range), flags=['multi_index']):
    init_gamma = np.array([gamma_values[0]])
    init_beta = np.array([gamma_values[1]])

    # QAOA_proxy_run を実行
    result_proxy = QAOA_proxy_run(
        num_constraints=num_constraints,
        num_qubits=num_qubits,
        p=p,
        init_gamma=init_gamma,
        init_beta=init_beta,
        expectations=example_expectations
    )
    new_proxy_expectation = result_proxy["expectation"]

    # QAOA_paper_proxy_run を実行
    result_paper = QAOA_paper_proxy_run(
        num_constraints=num_constraints,
        num_qubits=num_qubits,
        p=p,
        init_gamma=init_gamma,
        init_beta=init_beta,
        expectations=example_expectations
    )
    paper_proxy_expectation = result_paper["expectation"]

    # 結果を保存
    all_results.append({
        "init_gamma": init_gamma,
        "init_beta": init_beta,
        "new_proxy_expectation": new_proxy_expectation,
        "paper_proxy_expectation": paper_proxy_expectation
    })
    

# 結果のリストからデータを抽出
gamma_vals = [result["init_gamma"][0] for result in all_results]
beta_vals = [result["init_beta"][0] for result in all_results]
new_proxy_expecs = [result["new_proxy_expectation"] for result in all_results]
paper_proxy_expecs = [result["paper_proxy_expectation"] for result in all_results]

# 3Dプロットの作成
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# new_proxy_expectationのプロット
sc = ax.scatter(gamma_vals, beta_vals, new_proxy_expecs, c='r', marker='o', label='new_proxy_expectation')

# paper_proxy_expectationのプロット
ax.scatter(gamma_vals, beta_vals, paper_proxy_expecs, c='b', marker='x', label='paper_proxy_expectation')

# 軸ラベルの設定
ax.set_xlabel('Init Gamma')
ax.set_ylabel('Init Beta')
ax.set_zlabel('Expectation Value')
ax.legend()

plt.show()





ValueError: n must be a non-negative integer